# Automated ML

we start by importing all the dependencies that we will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Workspace, Environment
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
import json
import joblib


## Dataset

### Overview
We first of all define the workspace and create the experiment 'Parkinson-classification-AutoML' 

We then access to the dataset from the external link (placed in the github)

We move then to prepare the data(read the data using TabularDatasetFactory  then transforme it to a dataframe then store it in a csv file ) and send it to the default datastore so that we can use it in the Automl Config 

To finally create the compute cluster 

The dataset used in this notebook  is composed of a range of biomedical voice measurements from 31 people, 23 with Parkinson's disease (PD). Each column in the data is a particular voice measure, and each row corresponds one of 195 voice recording from these individuals ("name" column)

In [2]:
#Connection to the workspace and definition of the experiment 
ws = Workspace.from_config()
experiment_name = 'Parkinson-classification-AutoML'

experiment=Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-132680
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-132680


In [3]:
#Get the data from github
path_url = 'https://raw.githubusercontent.com/hananeouhammouch/Parkinsons-detection/master/parkinsons.data'
ds = TabularDatasetFactory.from_delimited_files(path = path_url)
ds.to_pandas_dataframe().head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [4]:
#Prepare the data and send it to the datastore
ds.to_pandas_dataframe().drop("name", inplace=True, axis=1)
ds.to_pandas_dataframe().to_csv("./training_data.csv")
datastore = ws.get_default_datastore()
datastore.upload(src_dir = "./", target_path = "data/")
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_data.csv"))])


Uploading an estimated of 11 files
Target already exists. Skipping upload for data/automl.ipynb
Target already exists. Skipping upload for data/automl.log
Target already exists. Skipping upload for data/azureml_automl.log
Target already exists. Skipping upload for data/hyperparam_tuning.ipynb
Target already exists. Skipping upload for data/train.py
Target already exists. Skipping upload for data/training_dataset.csv
Target already exists. Skipping upload for data/.ipynb_aml_checkpoints/automl-checkpoint2020-11-31-18-35-47.ipynb
Target already exists. Skipping upload for data/.ipynb_aml_checkpoints/hyperparam_tuning-checkpoint2020-11-31-18-33-20.ipynb
Target already exists. Skipping upload for data/.ipynb_checkpoints/automl-checkpoint.ipynb
Target already exists. Skipping upload for data/.ipynb_checkpoints/hyperparam_tuning-checkpoint.ipynb
Uploading ./training_data.csv
Uploaded ./training_data.csv, 1 files out of an estimated total of 6
Uploaded 1 files


In [5]:
#Creation of the compute-cluster
amlcompute_cluster_name = "cpu-cluster"

try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=5)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True , min_node_count = 1, timeout_in_minutes = 2)

Creating
Succeeded....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

In this section we configure the automnl setting and submit the experiment 

And Bellow is the description/Reason for the configuration used in Automl

|Setting |Reasons ?|
|-|-|
|**experiment_timeout_minutes**|Maximum amount of time in minutes that all iterations combined can take before the experiment terminates (15 minute because the dataset include only 195 lines)|
|**max_concurrent_iterations**|To help manage child runs  in parallele mode and when they can be performed, we create a dedicated cluster per experiment, and match the number of this setting (4) to the number of nodes in the cluster(5-1))|
|**n_cross_validations**|Number of cross validation (5) splits to ensure that they will be no overfiting |
|**primary_metric**|This is the metric that we want to optimize (accuracy) |
|**task**|classification |
|**compute_target**|To define the compute cluster we will be using |
|**training_data**|To specify the training dataset stored in the datastore  |
|**label_column_name**|To specify the dependent variable that we are trying to classify |

In [6]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes" :15,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}

# Automl config 
automl_config = AutoMLConfig(
    task="classification",
    compute_target=aml_compute,
    training_data=training_data,
    label_column_name="status",
    **automl_settings
)

In [7]:
# Submit experiment
auto_ml_run = experiment.submit(config = automl_config, show_output = True)


Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_d641fccc-1da4-4006-977e-0584f94ac0d7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: 

## Run Details

`RunDetails` widgets used here to show the different experiments.

In [8]:
RunDetails(auto_ml_run).show()
auto_ml_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_d641fccc-1da4-4006-977e-0584f94ac0d7',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-31T18:52:10.658636Z',
 'endTimeUtc': '2020-12-31T19:17:48.295891Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Parkinson-classification-AutoML","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-132680","workspace_name":"quick-starts-ws-132680","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"nu

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

The best model from the automl experiments is retrieved and  all the properties of the model are displayed



In [9]:
# Retrieve the best automl model.
best_run, fitted_model = auto_ml_run.get_output()
print(best_run)
print(fitted_model)

#Register the model
description ='Parkinson detection classification '
model_name='Parkinson-detection-automl'
model_path='./'
tags = None
model = auto_ml_run.register_model(model_name = model_name, description = description , tags = tags)
print(auto_ml_run.model_id)

Run(Experiment: Parkinson-classification-AutoML,
Id: AutoML_d641fccc-1da4-4006-977e-0584f94ac0d7_55,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               objective='reg:logistic',
                                                                                               random_state=0,
               

## Model Deployment


In the cell below we first  register the model, then define the environement to finally  create the inference config and deploy the model as a web service.

In [19]:
#Save the best model for the deployement
import os
os.makedirs('./amlmodel', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./amlmodel','automl_best_model_park.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./amlmodel', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [22]:
#Register the best model for the deployement

model=best_run.register_model(
            model_name = 'automl-bestmodel-parkinson', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description='Parkinson detection'
)

In [23]:
# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [24]:
# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_auto_park.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_auto_park.py',
                                    environment=myenv)

In [25]:
# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [26]:
# deploye the model a a web service 
service = Model.deploy(workspace=ws, 
                       name='automl-bestmodel-parkinso', 
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

In [27]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-132680', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-132680'), name=automl-bestmodel-parkinso, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [28]:
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)



Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state:
Healthy
Scoring URI:
http://fe9af3c7-a0c1-4152-9e22-0a3dfb91339d.southcentralus.azurecontainer.io/score
Swagger URI:
http://fe9af3c7-a0c1-4152-9e22-0a3dfb91339d.southcentralus.azurecontainer.io/swagger.json


In [ ]:
x_ds= ds.to_pandas_dataframe().drop("name", inplace=True, axis=1).sample(2)
y_ds = x_ds.pop('status')

Send a request to the web deployed service  to test it.

In the cell below, the logs of the web service are printed once over the service is deleted

In [ ]:
service_local.delete()
aml_compute.delete()